In [ ]:
import numpy as np
import os
import cv2
import math
import tarfile
from tqdm import tqdm

In [ ]:
def msgtobinary(msg):
    if type(msg) == str:
      # ord for ascii code
        result= ''.join([ format(ord(i), "08b") for i in msg ])
    
    # check if in bytes or check if the msg is the n dimensional (nd) array of pixels
    elif type(msg) == bytes or type(msg) == np.ndarray:
        result= [ format(i, "08b") for i in msg ]
    
    # **
    elif type(msg) == int or type(msg) == np.uint8:
        result= format(msg, "08b")

    else:
        raise TypeError("Input type is not supported in this function")
    
    return result

In [ ]:
def encrypt(s,n):
  fence = [[] for i in range(n)]
  rail  = 0
  var   = 1

  for char in s:
    fence[rail].append(char)
    rail += var

    if rail == n-1 or rail == 0:
      var = -var

  res = ''
  for i in fence:
    for j in i:
      res += j

  return res

In [ ]:
def encode_img_data(img):
    data=input("\nEnter the data to be Encoded in Image :")    
    if (len(data) == 0): 
        raise ValueError('Data entered to be encoded is empty')
  
    nameoffile = input("\nEnter the name of the New Image (Stego Image) after Encoding(with extension):")
    
    # rows*columns*3(for rgb)/ 8(bit size)
    no_of_bytes=(img.shape[0] * img.shape[1] * 3) // 8
    
    print("\t\nMaximum bytes to encode in Image :", no_of_bytes)
    encryptedData = encrypt(data,6)
    encryptedData +='*^*^*'
    
    if(len(data)>no_of_bytes):
        raise ValueError("Insufficient bytes Error, Need Bigger Image or give Less Data !!")    
    
    binary_data=msgtobinary(encryptedData)
    print("\n")
    print(binary_data)
    print("\n")
    print(encryptedData)

    length_data=len(binary_data)
    
    print("\nThe Length of Binary data",length_data)
    
    index_data = 0
    
    for i in img:
        for pixel in i:
            r, g, b = msgtobinary(pixel)
            if index_data < length_data:
                pixel[0] = int(r[:-1] + binary_data[index_data], 2) 
                index_data += 1
            if index_data < length_data:
                pixel[1] = int(g[:-1] + binary_data[index_data], 2) 
                index_data += 1
            if index_data < length_data:
                pixel[2] = int(b[:-1] + binary_data[index_data], 2) 
                index_data += 1
            if index_data >= length_data:
                break
    cv2.imwrite(nameoffile,img)
    print("\nEncoded the data successfully in the Image and the image is successfully saved with name ",nameoffile)

In [ ]:
def compress(tar_file, members):
    """
    Adds files (`members`) to a tar_file and compress it
    """
    # open file for gzip compressed writing
    tar = tarfile.open(tar_file, mode="w:gz")
    # with progress bar
    # set the progress bar
    progress = tqdm(members)
    for member in progress:
        # add file/folder/link to the tar file (compress)
        tar.add(member)
        # set the progress description of the progress bar
        progress.set_description(f"Compressing {member}")
    # close the file
    tar.close()

In [ ]:
def decrypt(s,n):
    fence = [[] for i in range(n)]
    rail  = 0
    var   = 1

    for char in s:
        fence[rail].append(char)
        rail += var

        if rail == n-1 or rail == 0:
            var = -var

    rFence = [[] for i in range(n)]

    i = 0
    l = len(s)
    s = list(s)
    for r in fence:
        for j in range(len(r)):
            rFence[i].append(s[0])
            s.remove(s[0])
        i += 1

    rail = 0
    var  = 1
    r = ''
    for i in range(l):
        r += rFence[rail][0]
        rFence[rail].remove(rFence[rail][0])
        rail += var

        if rail == n-1 or rail == 0:
            var = -var
    return r

In [ ]:
def decode_img_data(img):
    data_binary = ""
    for i in img:
        for pixel in i:
            r, g, b = msgtobinary(pixel) 
            data_binary += r[-1]  
            data_binary += g[-1]  
            data_binary += b[-1] 
            # ** 
            total_bytes = [ data_binary[i: i+8] for i in range(0, len(data_binary), 8) ]
            decoded_data = ""
            for byte in total_bytes:
                decoded_data += chr(int(byte, 2))
                if decoded_data[-5:] == "*^*^*": 
                    print("\n\nThe Encoded data which was hidden in the Image was :--  ",decrypt(decoded_data[:-5],6))
                    return 

In [ ]:
def decompress(tar_file, path, members=None):
    """
    Extracts `tar_file` and puts the `members` to `path`.
    If members is None, all members on `tar_file` will be extracted.
    """
    tar = tarfile.open(tar_file, mode="r:gz")
    if members is None:
        members = tar.getmembers()
    # with progress bar
    # set the progress bar
    progress = tqdm(members)
    for member in progress:
        tar.extract(member, path=path)
        # set the progress description of the progress bar
        progress.set_description(f"Extracting {member.name}")
    # or use this
    # tar.extractall(members=members, path=path)
    # close the file
    tar.close()

In [ ]:
def img_steg():
    while True:
        print("\n\t\tIMAGE STEGANOGRAPHY OPERATIONS\n") 
        print("1. Encode the Text message") 
        print("2. Decode the Text message")
        print("3. Compress the Image")
        print("4. Decompress the Image") 
        print("5. Exit")  
        choice1 = int(input("Enter the Choice: "))   
        if choice1 == 1:
            image=cv2.imread("/content/sample_data/image2.jpg")
            encode_img_data(image)
        elif choice1 == 2:
            # image1=cv2.imread(input("Enter the Image you need to Decode to get the Secret message :  "))
            image1 = cv2.imread("/content/extracted/tar_file/encode.png")
            decode_img_data(image1)
        elif choice1 == 3:
            compress("compressed.tar.gz", ["tar_file"])
        elif choice1 == 4:
            decompress("compressed.tar.gz", "extracted")
        elif choice1 == 5:
            break
        else:
            print("Incorrect Choice")
        print("\n")

In [ ]:
def main():
    print("\t\t      STEGANOGRAPHY")   
    while True:  
        print("\n\t\t\tMAIN MENU\n")  
        print("1. IMAGE STEGANOGRAPHY {Hiding Text in Image cover file}")  
        print("2. Exit\n")  
        choice1 = int(input("Enter the Choice: "))   
        if choice1 == 1: 
            img_steg()
        elif choice1 == 2:
            break
        else:
            print("Incorrect Choice")
        print("\n\n")

In [ ]:
if __name__ == "__main__":
    main()

		      STEGANOGRAPHY

			MAIN MENU

1. IMAGE STEGANOGRAPHY {Hiding Text in Image cover file}
2. Exit

Enter the Choice: 1

		IMAGE STEGANOGRAPHY OPERATIONS

1. Encode the Text message
2. Decode the Text message
3. Compress the Image
4. Decompress the Image
5. Exit
Enter the Choice: 3


Compressing tar_file: 100%|██████████| 1/1 [00:01<00:00,  1.65s/it]





		IMAGE STEGANOGRAPHY OPERATIONS

1. Encode the Text message
2. Decode the Text message
3. Compress the Image
4. Decompress the Image
5. Exit


KeyboardInterrupt: ignored